In [2]:
import numpy as np
import pandas as pd
import sqlite3
import optuna
import time
from scipy.optimize import curve_fit
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator
import japanize_matplotlib
from IPython.display import display, HTML
from epics import caput


/home/skato/anaconda3/envs/optuna/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# CSVファイルを読み込む
df = pd.read_csv('5_23_30min.csv', na_values=['N/A'])

# N/Aを前の行の値で埋める
df.ffill(inplace=True)

# Timestamp列（ここでは一列目を指定）をdatetimeオブジェクトに変換してdfに追加
df['timestamp'] = pd.to_datetime(df.iloc[:,0])

# Unix時間（秒）に変換してdfに追加
df['unix_time'] = df['timestamp'].astype('int64') // 10**9

df.to_csv('check.csv', index=False)

# BPMのchargeを電流値に変換してdfに追加
#df["BPM_to_current"] = (df["CG_BTP:BPM:CHARGE1"]-0.1) * 0.1

In [4]:
# 必要なデータを抽出
timestamps = pd.to_datetime(df['timestamp'])
data_values = {
    "B2_VXD:Rad:Res:BPAtDCU3:A1:DoseRate": df["B2_VXD:Rad:Res:BPAtDCU3:A1:DoseRate"].values,
    "TM_EVR0:HER:INJ2_EFF:BCM": df["TM_EVR0:HER:INJ2_EFF:BCM"].values,
    "CGHINJ:BEAM_GATE:STATUS": df["CGHINJ:BEAM_GATE:STATUS"].values,
    "BTeBPM:QMD8E_M_1:NC_1Hz:C": df["BTeBPM:QMD8E_M_1:NC_1Hz:C"].values,
    "BMHDCCT:CURRENT": df["BMHDCCT:CURRENT"].values,
}

# EPICS PV 名を設定
pv_names = {
    "B2_VXD:Rad:Res:BPAtDCU3:A1:DoseRate": 'TEST:LM0',
    "TM_EVR0:HER:INJ2_EFF:BCM": 'TEST:LM1',
    "CGHINJ:BEAM_GATE:STATUS": 'TEST:TH0',
    "BTeBPM:QMD8E_M_1:NC_1Hz:C": 'TEST:TH1',
    "BMHDCCT:CURRENT": 'TEST:LM2',
}

# データを書き込む関数
def write_to_pv(pv_names, timestamps, data_values):
    start_time = time.time()  # スクリプト開始時の時間を記録
    initial_time = timestamps[0]  # データの最初のタイムスタンプを基準にする
    
    # 初回データポイントをスキップするためにインデックスを 1 から開始
    for i in range(1, len(timestamps)):
        timestamp = timestamps[i]
        # 現在の時間とスクリプト開始時の差を計算
        elapsed_real_time = time.time() - start_time
        # データのタイムスタンプに基づく経過時間を計算
        elapsed_data_time = (timestamp - initial_time).total_seconds()
        
        # 次のデータポイントの時間まで待機
        if elapsed_data_time > elapsed_real_time:
            wait_time = elapsed_data_time - elapsed_real_time
            print(f'Waiting for {wait_time} seconds')
            time.sleep(wait_time)
            
        else:
            wait_time = 0
            
        # PV に値を書き込む
        for key, values in data_values.items():
            pv_name = pv_names[key]
            value = values[i]
            caput(pv_name, value)
            print(f'Wrote {value} to {pv_name} at data time {timestamp} (elapsed data time: {elapsed_data_time} seconds, elapsed real time: {elapsed_real_time} seconds)')
        
        print(f'Waited for {wait_time} seconds, proceeding to next timestamp.')

# 関数を実行
write_to_pv(pv_names, timestamps, data_values)





Waiting for 2.090964475631714 seconds
Wrote nan to TEST:LM0 at data time 2024-05-23 23:29:59.219000 (elapsed data time: 2.091 seconds, elapsed real time: 3.552436828613281e-05 seconds)
cannot connect to TEST:BCM
Wrote nan to TEST:BCM at data time 2024-05-23 23:29:59.219000 (elapsed data time: 2.091 seconds, elapsed real time: 3.552436828613281e-05 seconds)
cannot connect to TEST:STATUS
Wrote 1.0 to TEST:STATUS at data time 2024-05-23 23:29:59.219000 (elapsed data time: 2.091 seconds, elapsed real time: 3.552436828613281e-05 seconds)
cannot connect to TEST:NC_1Hz
Wrote nan to TEST:NC_1Hz at data time 2024-05-23 23:29:59.219000 (elapsed data time: 2.091 seconds, elapsed real time: 3.552436828613281e-05 seconds)
cannot connect to TEST:CURRENT
Wrote 997.618 to TEST:CURRENT at data time 2024-05-23 23:29:59.219000 (elapsed data time: 2.091 seconds, elapsed real time: 3.552436828613281e-05 seconds)
Waited for 2.090964475631714 seconds, proceeding to next timestamp.
Wrote nan to TEST:LM0 at da